# Weights for nn4

In [1]:
import tensorflow as tf
from keras import backend as K
from keras.utils import CustomObjectScope
from keras.models import load_model
import keras

Using TensorFlow backend.


In [2]:
tf.__version__, keras.__version__

('1.13.1', '2.2.4')

In [3]:
sess = tf.Session()
K.set_session(sess)
import cv2

In [4]:
with CustomObjectScope({'tf': tf}):
    model = load_model('/home/coep/pkm/CNN-distillation/py3model/keras/nn4.small2.lrn.h5')
    
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 102, 102, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 48, 48, 64)   9472        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
bn1 (BatchNormalization)        (None, 48, 48, 64)   256         conv1[0][0]                      
_____________________________________

/home/coep/venv/tflow-gpupy3/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
help(model)

Help on Model in module keras.engine.training object:

class Model(keras.engine.network.Network)
 |  The `Model` class adds training & evaluation routines to a `Network`.
 |  
 |  Method resolution order:
 |      Model
 |      keras.engine.network.Network
 |      keras.engine.base_layer.Layer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  compile(self, optimizer, loss=None, metrics=None, loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None, **kwargs)
 |      Configures the model for training.
 |      
 |      # Arguments
 |          optimizer: String (name of optimizer) or optimizer instance.
 |              See [optimizers](/optimizers).
 |          loss: String (name of objective function) or objective function.
 |              See [losses](/losses).
 |              If the model has multiple outputs, you can use a different loss
 |              on each output by passing a dictionary or a list of losses.
 |              The loss value

In [6]:
model.weights

[<tf.Variable 'conv1/kernel:0' shape=(7, 7, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv1/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'bn1/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'bn1/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv2/kernel:0' shape=(1, 1, 64, 64) dtype=float32_ref>,
 <tf.Variable 'conv2/bias:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'bn2/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'bn2/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'conv3/kernel:0' shape=(3, 3, 64, 192) dtype=float32_ref>,
 <tf.Variable 'conv3/bias:0' shape=(192,) dtype=float32_ref>,
 <tf.Variable 'bn3/gamma:0' shape=(192,) dtype=float32_ref>,
 <tf.Variable 'bn3/beta:0' shape=(192,) dtype=float32_ref>,
 <tf.Variable 'inception_3a_3x3_conv1/kernel:0' shape=(1, 1, 192, 96) dtype=float32_ref>,
 <tf.Variable 'inception_3a_3x3_conv1/bias:0' shape=(96,) dtype=float32_ref>,
 <tf.Variable 'inception_3a_5x5_conv1/kernel:0' shape=(1, 1, 192, 16) dty

In [41]:
model.layers

In [42]:
len(model.layers)

166

In [43]:
#April 10 2019
import numpy as np
f=open("demofile.h","a");
f.write("float weights[");
a=100;
f.write(str(a))
a=np.array([1,2,3,4,5]);
np.savetxt(f,[a],fmt='%.6e',delimiter=",")
f.write("};")
f.close();

In [348]:
temp="inception_5b"
file_name="nn4_weights_"+temp;
f=open(file_name+".h","w");
f.close();

In [362]:
num="";
main=temp+"_1x1"
main_conv=main+"_conv"+num
main_bn=main+"_bn"+num

In [363]:
#April 10 2019
#for conv weights
count=0;
import numpy as np
layer_name=main_conv
f=open(file_name+".h","a");
s=model.get_layer(layer_name).get_weights()[0].shape
size=s[0]*s[1]*s[2]*s[3];
f.write("float filter_"+layer_name+"["+str(size)+"] = {");
for k in range(0,s[3]):
    for i in range(0,s[0]):
        for j in range(0,s[1]):
            a=np.asarray(model.get_layer(layer_name).get_weights()[0][i,j,:,k]);
            for l in range(0,a.shape[0]):
                count=count+1;
                f.write(str(a[l]));
                if(count!=size):
                    f.write(',');
f.write('};\n\n')
f.close();
print(count)

188416


In [364]:
#April 10 2019
#for conv biases
import numpy as np
layer_name=main_conv
f=open(file_name+".h","a");
s=model.get_layer(layer_name).get_weights()[1].shape
size=s[0];
f.write("float bias_"+layer_name+"["+str(size)+"] = {");
a=np.asarray(model.get_layer(layer_name).get_weights()[1]);
for l in range(0,a.shape[0]):
    f.write(str(a[l]));
    if(l!=a.shape[0]-1):
        f.write(',');
f.write('};\n\n')
f.close();

In [365]:
#April 10 2019
#for batch norm layers weights order is : [gamma, beta, mean, std]
import numpy as np
names=['gamma','beta','mean','std_dev'];
layer_name=main_bn
f=open(file_name+".h","a");
for layer_num in range(0,4):
    s=model.get_layer(layer_name).get_weights()[layer_num].shape
    size=s[0];
    f.write("float "+str(names[layer_num])+'_'+layer_name+"["+str(size)+"] = {");
    a=np.asarray(model.get_layer(layer_name).get_weights()[layer_num]);
    if(layer_num==3):
        a=a+0.00001;
        a=np.sqrt(a)
    for l in range(0,a.shape[0]):
        f.write(str(a[l]));
        if(l!=a.shape[0]-1):
            f.write(',');
    f.write('};\n\n')

f.close();

In [353]:
import numpy as np;
name='inception_3a_1x1_bn';
s=model.get_layer(name).get_weights()[3];
shape=s.shape
a=np.array([]);
for i in range(0,shape[0]):
    a=np.append(a,np.asarray(np.sqrt(s[i]+0.00001)));
a.tofile(name+'_std_dev.csv',sep=',');

In [54]:
name="inception_5b_1x1_conv"
f_handle = open(name+'.h', 'a')
s=model.get_layer(name);
print(simport cv2.trainable)
s=s.get_weights()[0].shape
import numpy as np;
a=np.array([]);
for k in range(0,s[3]):
    for i in range(0,s[0]):
        for j in range(0,s[1]):
            a=np.append(a,np.asarray(model.get_layer(name).get_weights()[0][i,j,:,k]))
            f_handle.write(str(a))

#np.savetxt(f_handle, a)
f_handle.close()

True


In [48]:
import numpy as np;
name="inception_5b_1x1_conv"
s=model.get_layer(name).get_weights()[0].shape
for k in range(0,s[3]):
    for i in range(0,s[0]):
        for j in range(0,s[1]):
            a=model.get_layer(name).get_weights()[0][i,j,:,k]

-0.003167807

In [ ]:
name="inception_5b_1x1_conv"
s=model.get_layer(name).get_weights()[0].shape
import numpy as np;
a=np.array([]);
for k in range(0,s[3]):
    for i in range(0,s[0]):
        for j in range(0,s[1]):
            a=np.append(a,np.asarray(model.get_layer(name).get_weights()[0][i,j,:,k]))
a.tofile(name+'_weights.csv',sep=',');
a.shape


In [205]:
a=np.asarray(model.get_layer(name).get_weights()[1]);
a.tofile(name+'_biases.csv',sep=',');
a.shape

(128,)

In [204]:
name='dense_layer'
s=model.get_layer(name).get_weights()[0].shape
a=np.array([]);
for i in range(0,s[0]):
    a=np.append(a,np.asarray(model.get_layer(name).get_weights()[0][i,:]))
a.tofile(name+'_weights.csv',sep=',');
a.shape

(94208,)

In [207]:
name="conv1"
model.get_layer(name).get_weights()[0].shape

(7, 7, 3, 64)

In [7]:
model.layers[-2].get_weights()

[array([[ 0.0638466 , -0.00509637, -0.05529933, ..., -0.06065852,
          0.03162977,  0.03039331],
        [ 0.0359335 , -0.04079844, -0.05175897, ..., -0.09534544,
         -0.1398415 , -0.02489908],
        [ 0.00336699, -0.01912811, -0.03294028, ...,  0.01073863,
          0.02246026,  0.04493647],
        ...,
        [-0.00241918, -0.01574283, -0.03267112, ...,  0.01423741,
          0.0355286 ,  0.00118524],
        [-0.0324402 , -0.05209239, -0.01657831, ...,  0.0103431 ,
          0.00325075, -0.0138299 ],
        [ 0.04424537, -0.03503973, -0.03601842, ...,  0.09149736,
         -0.06755237,  0.0689095 ]], dtype=float32),
 array([ 6.02469333e-02, -7.37190470e-02,  1.00949239e-02, -1.19365856e-01,
         6.82547167e-02,  1.05848305e-01,  3.09077632e-02,  2.75030341e-02,
        -5.02438359e-02, -3.46696042e-02, -2.14334540e-02,  6.68670461e-02,
         7.48417899e-02,  8.82615335e-03,  3.08016650e-02, -7.48884678e-02,
        -7.43789151e-02,  3.20000276e-02, -1.31938113e

# Mid layer outputs template

In [44]:
import numpy as np

In [8]:
model.get_weights()

[array([[[[ 1.49056152e-01, -1.34768113e-01,  1.51423007e-01, ...,
           -1.51435016e-02, -2.09911406e-01,  1.02350123e-01],
          [-5.29299909e-03, -1.72828153e-01,  6.74600080e-02, ...,
            9.49144084e-03, -7.36269802e-02, -2.80116163e-02],
          [-9.84221995e-02, -1.66645259e-01,  1.76576570e-01, ...,
           -8.98738280e-02, -3.16160247e-02,  2.74819811e-03]],
 
         [[ 3.11939176e-02, -6.52555674e-02, -1.57041132e-01, ...,
            6.28847480e-02,  1.02878734e-02, -2.26173317e-03],
          [-2.03805685e-01, -2.17020586e-01,  1.01853756e-03, ...,
            8.54506642e-02,  1.08859288e-02,  8.94757062e-02],
          [-1.10391408e-01, -2.36338690e-01, -9.43221152e-03, ...,
            4.73738909e-02, -5.14071323e-02, -6.60058260e-02]],
 
         [[ 8.75389948e-02,  1.17656374e-02, -1.05412856e-01, ...,
            9.44804549e-02, -2.67343465e-02, -4.21006000e-03],
          [ 2.84998678e-02, -2.90884197e-01,  8.19148645e-02, ...,
            1.889

In [11]:
#img = cv2.resize(cv2.imread('/home/coep/Downloads/sample.jpeg', 1), (96, 96))[..., ::-1].reshape(1, 96, 96, 3)
import numpy as np
img = cv2.resize(cv2.imread('/home/coep/Downloads/sample.jpeg', 1), (96, 96)).reshape(1, 96, 96, 3)
img = np.around(img/255.0, decimals=6)
feed_dict = {model.input: img, K.learning_phase(): 0}

with tf.device('/gpu:0'):
    with sess.as_default():
        outp = sess.run(model.get_layer("concatenate_7").output, feed_dict=feed_dict)
        print(outp.shape, type(outp))

(1, 3, 3, 736) <class 'numpy.ndarray'>


In [13]:
outp[0,:,:,10]

array([[0.        , 0.        , 0.        ],
       [0.        , 0.09552448, 0.        ],
       [0.        , 0.        , 0.        ]], dtype=float32)

In [30]:
import numpy as np
img = cv2.resize(cv2.imread('/home/coep/Downloads/sample.jpeg', 1), (96, 96)).reshape(1, 96, 96, 3)
img = np.around(img/255.0, decimals=6)
feed_dict = {model.input: img, K.learning_phase(): 0}

with tf.device('/gpu:0'):
    with sess.as_default():
        outp = sess.run(model.get_layer("concatenate_4").output, feed_dict=feed_dict)
        print(outp.shape, type(outp))

(1, 6, 6, 640) <class 'numpy.ndarray'>


In [31]:
outp[0,:,:,330]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.2850363 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.04753665, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]], dtype=float32)

In [28]:
img[0,0,0,2]

0.098039

In [10]:
layer_name='dense'
s=model.get_layer(layer_name).get_weights()[0].shape


ValueError: No such layer: dense

In [30]:
dense_weights=model.layers[-2].get_weights()[0]

In [46]:
dense_weights[1,:]

array([ 3.59334983e-02, -4.07984443e-02, -5.17589711e-02, -6.14243820e-02,
       -2.41376366e-02, -5.86759001e-02,  8.10842440e-02,  1.91693362e-02,
       -2.20971685e-02,  3.67821604e-02,  1.02277165e-02,  1.60858128e-02,
        2.50490401e-02, -8.54429305e-02,  3.22575122e-02,  2.08854172e-02,
       -2.06133369e-02,  4.54647318e-02,  1.99067574e-02, -8.44493732e-02,
        2.43153442e-02,  5.36442511e-02, -5.06287552e-02,  6.14446700e-02,
       -1.04203276e-01,  6.95806444e-02, -6.33496642e-02, -1.42302603e-01,
        5.09131793e-03,  4.11236398e-02, -4.46764044e-02,  1.78088829e-01,
       -2.56312918e-02,  8.27316716e-02,  3.97231393e-02, -8.10319185e-02,
        7.95527026e-02,  9.23510268e-02,  1.14958242e-01,  4.38104086e-02,
        5.80588058e-02, -3.32423719e-03,  4.70840558e-03,  5.13602421e-03,
        5.11622466e-02, -4.24024044e-03,  5.61608672e-02, -1.65954418e-02,
       -9.11465883e-02, -1.31813481e-01,  1.10153519e-02, -3.49516943e-02,
        5.68667874e-02,  

In [47]:
a=dense_weights.flatten()
np.save('dense_weights',a)

In [29]:
np.save('final_avg_pooling',outp)